# Getting Preciser Timestamps
* In proc 1 to 3, there were cases where GPT4.1 took longer than expected. 
* The structured logs in the JSONL file stored the start time when the method that calls the API was called and the end time when the response was received. 
* However, this did not account for the case of OpenAI's code doing the retries, so the retries where included in the time calculation.
* In this notebook, we use the logs to extract start and end times that are more in line with the real translation time
* We do this by looking at the `DEBUG` logs created by OpenAI's code, they contain an entry of when exactly a request was sent.
* The end time we can obtain by our own logs, looking for `Translated X sents for src-tgt` message.

In [41]:
!cat proc3.log | grep -P "(Sending HTTP Request: POST https://api.openai.com/v1/chat/completions|Translated \d+ sents for \w\w-\w\w)"

DEBUG: 2025-05-09 13:18:06 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
INFO: 2025-05-09 13:22:10 - [✔️]: Translated 400 sents for fr-da
DEBUG: 2025-05-09 13:22:10 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
INFO: 2025-05-09 13:26:42 - [✔️]: Translated 400 sents for fr-de
DEBUG: 2025-05-09 13:26:42 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG: 2025-05-09 13:31:43 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
INFO: 2025-05-09 13:36:34 - [✔️]: Translated 400 sents for fr-el
DEBUG: 2025-05-09 13:36:35 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
INFO: 2025-05-09 13:39:17 - [✔️]: Translated 400 sents for fr-es
DEBUG: 2025-05-09 13:39:17 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG: 2025-05-09 13:49:07 - Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG: 2025-05-09 13:54:08 - Sending HTTP Reque

In [42]:
import json
from os.path import join
from datetime import datetime



with open(join('tasks', 'proc3.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]

pairs = [('sv', 'es'), ('pt', 'da')]

for log in logs:
    pair = (log['src_lang'], log['tgt_lang'])
    if pair in pairs:
        print(pair)
        start = datetime.fromtimestamp(
            log['start']).strftime('%Y-%m-%d %H:%M:%S')
        end = datetime.fromtimestamp(log['end']).strftime('%Y-%m-%d %H:%M:%S')
        print('start', start)
        print('end', end)
        print('duration', log['end'] - log['start'])
        print()


('pt', 'da')
start 2025-05-09 19:19:38
end 2025-05-09 19:28:39
duration 541.1454193592072

('sv', 'es')
start 2025-05-09 21:21:34
end 2025-05-09 21:24:14
duration 160.7068531513214



* For some cases, the structured logs from `proc3.jsonl` got the time right because the automtic retries exceeded OpenAI's limit of 2 retries and triggered the retries implemented by us, which restarted the timing.
* However, in other cases, such as `pt-da`, we observe that the structured logs captured start time `19:19:38` which corresponds to the first try of OpenAI's code, not the last, hence the duration is longer than it really was.
* Our goal is to now to go through proc1 to 3 logs and store the real start and end times. We lose precision since the timestamps are not UNIX timestamps but since we work with seconds anyway, that should not be a big issue.

In [43]:
!cat proc2.log proc3.log | grep -P "(Sending HTTP Request: POST https://api.openai.com/v1/chat/completions|Translated \d+ sents for \w\w-\w\w)" | grep -P -B1 "(Translated \d+ sents for \w\w-\w\w)" > tmp_proc2-3.log

In [44]:
with open('tmp_proc2-3.log', 'r') as f:
    logs = [ln for ln in f if ln.startswith('DEBUG') or ln.startswith('INFO')]

In [45]:
import re
pat = r"Translated \d+ sents for (\w\w-\w\w)"
pair2log_idx = {}
for i, log in enumerate(logs):
    if log.startswith('INFO'):
        pair = re.search(pat, log).group(1)
        pair2log_idx[pair] = i

In [46]:
import time
pair2time = {}
time_pat = r"(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d)"
fmt = '%Y-%m-%d %H:%M:%S'
for pair in pair2log_idx:
    idx = pair2log_idx[pair]
    start = re.search(time_pat, logs[idx-1]).group(1)
    end = re.search(time_pat, logs[idx]).group(1)
    start = datetime.strptime(start, fmt)
    start_unix = time.mktime(start.timetuple())
    end = datetime.strptime(end, fmt)
    end_unix = time.mktime(end.timetuple())
    pair2time[pair] = {'start': start_unix, 'end': end_unix}


In [47]:
proc2_success = 44
proc3_success = 42
len(pair2time) == proc2_success + proc3_success

True

In [48]:
import json
with open('proc2-3-europarl-gpt.json', 'w') as f:
    json.dump(pair2time, f)

* For Proc1, we need to make it a bit smarter, since Proc1 involved EuroParl, Opus100 and Flores+ as well, so we cannot use the pair directly as the sole key.

In [49]:
!cat proc1.log | grep -P "(Sending HTTP Request: POST https://api.openai.com/v1/chat/completions|Translated \d+ sents for \w\w-\w\w)" | grep -P -B1 "(Translated \d+ sents for \w\w-\w\w)" > tmp_proc1.log

In [50]:
with open('tmp_proc1.log', 'r') as f:
    logs = [ln for ln in f if ln.startswith('DEBUG') or ln.startswith('INFO')]

In [51]:
import re
pat = r"Translated \d+ sents for (\w\w-\w\w)"
pair2log_idx = {}
for i, log in enumerate(logs):
    if log.startswith('INFO'):
        pair = re.search(pat, log).group(1)
        if pair not in pair2log_idx:
            pair2log_idx[pair] = [i]
        else:
            pair2log_idx[pair].append(i)

In [52]:
import time
pair2time = {}
time_pat = r"(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d)"
fmt = '%Y-%m-%d %H:%M:%S'
for pair in pair2log_idx:
    indices = pair2log_idx[pair]
    pair2time[pair] = []
    for idx in indices:
        if logs[idx-1].startswith('DEBUG'):
            start = re.search(time_pat, logs[idx-1]).group(1)
            end = re.search(time_pat, logs[idx]).group(1)
            start = datetime.strptime(start, fmt)
            start_unix = time.mktime(start.timetuple())
            end = datetime.strptime(end, fmt)
            end_unix = time.mktime(end.timetuple())
            pair2time[pair].append({'start': start_unix, 'end': end_unix})


In [53]:
check = [len(times) == 3 for times in pair2time.values()]
all(check)

True

* Proc 1 task ids for GPT4.1 are:
    * `9549e927-4f63-4205-82bb-5c7ccabfe943` europarl
    * `fdbf6190-d061-4154-ae94-d7b11199d043` flores_plus
    * `fa628bc1-4f85-446d-9167-1a8d99ccc493` opus-100

In [54]:
!cat proc1.log | grep -P "Starting task"

INFO: 2025-05-07 13:55:45 - [🏁]: Starting task 9549e927-4f63-4205-82bb-5c7ccabfe943 on commit 66ef922
INFO: 2025-05-07 14:57:36 - [🏁]: Starting task ab9fc42b-61af-4534-afd6-3710eb1dbc52 on commit 66ef922
INFO: 2025-05-07 15:01:53 - [🏁]: Starting task fa628bc1-4f85-446d-9167-1a8d99ccc493 on commit 66ef922
INFO: 2025-05-07 15:38:48 - [🏁]: Starting task 4c5ffc31-c043-414d-aba1-270f9967f078 on commit 66ef922
INFO: 2025-05-07 15:41:35 - [🏁]: Starting task fdbf6190-d061-4154-ae94-d7b11199d043 on commit 66ef922
INFO: 2025-05-07 16:39:58 - [🏁]: Starting task 88538d1d-11fc-4356-9153-b105d4e26e30 on commit 66ef922


* So we infer that the first one is EuroParl, the second is Opus100 and the third is FloresPlus

In [55]:
pair2time_modified = {}
for pair, times in pair2time.items():
    pair2time_modified[pair] = {'ep': times[0], 'opus': times[1], 'flores': times[2]}

In [56]:
with open('proc1-gpt.json', 'w') as f:
    json.dump(pair2time_modified, f)

* For Proc 4 to 6, we can rely on structured logs as there were no automatic retries from OpenAI's side anymore.
* We can still confirm as a sanity check.

In [57]:
!cat proc4.log | grep -P "(Sending HTTP Request: POST https://api.openai.com/v1/chat/completions|Translated \d+ sents for \w\w-\w\w)" | grep -P -B1 "(Translated \d+ sents for \w\w-\w\w)" > tmp_proc4.log

In [58]:
with open('tmp_proc4.log', 'r') as f:
    logs = [ln for ln in f if ln.startswith('DEBUG') or ln.startswith('INFO')]

In [59]:
import re
pat = r"Translated \d+ sents for (\w\w-\w\w)"
pair2log_idx = {}
for i, log in enumerate(logs):
    if log.startswith('INFO'):
        pair = re.search(pat, log).group(1)
        pair2log_idx[pair] = i

In [60]:
import time
pair2time = {}
time_pat = r"(\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d)"
fmt = '%Y-%m-%d %H:%M:%S'
for pair in pair2log_idx:
    idx = pair2log_idx[pair]
    start = re.search(time_pat, logs[idx-1]).group(1)
    end = re.search(time_pat, logs[idx]).group(1)
    start = datetime.strptime(start, fmt)
    start_unix = time.mktime(start.timetuple())
    end = datetime.strptime(end, fmt)
    end_unix = time.mktime(end.timetuple())
    pair2time[pair] = {'start': start_unix, 'end': end_unix}

In [61]:
len(pair2time)

90

In [62]:
with open(join('tasks', 'proc4.jsonl'), 'r') as f:
    logs = [json.loads(ln) for ln in f.readlines()]
len(logs)

90

In [63]:
pair_keys_1 = set(pair2time.keys())
pair_keys_2 = set([f'{log["src_lang"]}-{log["tgt_lang"]}' for log in logs])
pair_keys_1 == pair_keys_2

True

In [64]:
diffs = []
for log in logs:
    pair = (log['src_lang'], log['tgt_lang'])
    pair_key = '-'.join(pair)
    structured_dur = log['end'] - log['start']
    unstructured_dur = pair2time[pair_key]['end'] - pair2time[pair_key]['start']
    diff = structured_dur - unstructured_dur
    diffs.append(diff)

print(f'Max diff: {max(diffs):.2f} s')
print(f'Min diff: {min(diffs):.2f} s')
print(f'Mean diff: {sum(diffs)/len(diffs):.2f} s')

Max diff: 0.79 s
Min diff: -1.21 s
Mean diff: -0.19 s


* Not too tragic

In [65]:
from random import sample
selected_pairs = sample(list(pair_keys_1), 5)
for log in logs:
    pair = (log['src_lang'], log['tgt_lang'])
    pair_key = '-'.join(pair)
    if pair_key in selected_pairs:
        print(pair_key)
        print('duration (structured log)', f'{log["end"] - log["start"]:.2f} s')
        print('duration (unstructured log)', f'{pair2time[pair_key]["end"] - pair2time[pair_key]["start"]:.2f} s')
        print()


el-fr
duration (structured log) 203.13 s
duration (unstructured log) 203.00 s

es-sv
duration (structured log) 192.00 s
duration (unstructured log) 192.00 s

it-da
duration (structured log) 160.44 s
duration (unstructured log) 161.00 s

pt-nl
duration (structured log) 150.94 s
duration (unstructured log) 151.00 s

sv-el
duration (structured log) 261.04 s
duration (unstructured log) 261.00 s



## Preparing Files for Analysis
* At the moment, all files are stored hierarchically in the `tasks` folder
* This is not very convenient for analysis purposes, so we stored them all into a single folder preserve hierarchical information by filename prefix.

In [3]:
import os
from os.path import join
import re
import json

procs = [f'proc{i}' for i in range(1, 7)]

long2short = {
    'gpt-4.1-2025-04-14': 'gpt',
    'deepl_document': 'deepl',
    'opus-100': 'opus',
    'europarl': 'ep',
    'flores_plus': 'flores'
}

prefix2file = {}
for p in procs:
    folder = join('tasks', p)
    folders = os.listdir(folder)
    for f in folders:
        tl_folders = os.listdir(join(folder, f))
        for t in tl_folders:
            tl_files = [f for f in os.listdir(join(folder, join(f, t))) if f.endswith('.txt')]
            for tf in tl_files:
                pair = tf.replace('.txt', '')
                if 'fail' in pair:
                    pair = re.sub(r'_fail\d+', '', pair)
                if p in ['proc1', 'proc2', 'proc5', 'proc6']:
                    sf = long2short[f]
                else:
                    sf = long2short[f.split('-')[0]]
                st = long2short[t]
                prefix = f'{sf}-{st}-{pair}'
                prefix2file[prefix] = {'file': join(folder, join(f, join(t, tf))), 'procedure': p}
                with open(join('tasks', f'{p}.jsonl'), 'r') as fi:
                    logs = [json.loads(ln) for ln in fi.readlines()]
                for log in logs:
                    log_pair = f'{log["src_lang"]}-{log["tgt_lang"]}'
                    if log_pair == pair:
                        prefix2file[prefix]['log'] = log


In [4]:
opus = 0
flores = 0
ep = 0
for prefix, info in prefix2file.items():
    p = prefix.split('-')[0]
    if 'opus' == p:
        opus += 1
    if 'flores' == p:
        flores += 1
    if 'ep' == p:
        ep += 1

print(opus, flores, ep)

40 220 219


* 40 translation for OPUS, 20 DeepL, 20 GPT4.1
* 220 translations for Flores+, 110 DeepL, 110 GPT4.1
* 219 translations for Europarl, 110 DeepL, 109 GPT4.1 (because it kept failing `it-el`)

In [5]:
import shutil
os.makedirs('translations', exist_ok=True)
for prefix, info in prefix2file.items():
    shutil.copy(info['file'], join('translations', f'{prefix}.txt'))

with open(join('translations', 'info.json'), 'w') as f:
    json.dump(prefix2file, f)

In [11]:
check = os.listdir('translations')
len(check) == 480

True

## Time Analysis

In [20]:
import json
prefix2time = {}
with open(join('translations', 'info.json'), 'r') as f:
    prefix2file = json.load(f)

for prefix, info in prefix2file.items():
    prefix2time[prefix] = info['log']['end'] - info['log']['start']

In [21]:
with open('proc2-3-europarl-gpt.json') as f:
    pair2time = json.load(f)

for prefix in prefix2time:
    if prefix.startswith('ep-gpt'):
        info = prefix2file[prefix]
        if info['procedure'] == 'proc2' or info['procedure'] == 'proc3':
            pair = '-'.join([prefix.split('-')[2], prefix.split('-')[3]])
            prefix2time[prefix] = pair2time[pair]['end'] - pair2time[pair]['start']


In [22]:
with open('proc1-gpt.json') as f:
    pair2time = json.load(f)

for prefix in prefix2time:
    data, tl = prefix.split('-')[:2]
    if tl == 'gpt':
        info = prefix2file[prefix]
        if info['procedure'] == 'proc1':
            pair = '-'.join([prefix.split('-')[2], prefix.split('-')[3]])
            prefix2time[prefix] = pair2time[pair][data]['end'] - pair2time[pair][data]['start']

In [26]:
times = []
for prefix in sorted(prefix2time, key=lambda x: prefix2time[x], reverse=True):
    if 'gpt' in prefix:
        print(prefix, f'{prefix2time[prefix]:.2f}')
        times.append(prefix2time[prefix])

ep-gpt-fi-nl 301.00
flores-gpt-en-el 299.00
ep-gpt-it-sv 299.00
flores-gpt-nl-el 295.98
flores-gpt-da-fi 295.43
ep-gpt-nl-sv 293.00
ep-gpt-fr-el 291.00
ep-gpt-pt-fi 291.00
ep-gpt-fr-sv 290.00
ep-gpt-nl-fi 290.00
ep-gpt-fr-nl 289.00
ep-gpt-pt-el 289.00
flores-gpt-el-fi 288.84
ep-gpt-da-fi 288.00
ep-gpt-it-fi 285.98
flores-gpt-fr-de 284.79
ep-gpt-el-da 281.00
flores-gpt-de-fr 280.56
ep-gpt-da-el 279.00
ep-gpt-el-it 279.00
ep-gpt-sv-fi 279.00
ep-gpt-en-fi 275.00
flores-gpt-it-el 274.76
ep-gpt-fr-fi 273.00
ep-gpt-fr-de 272.00
ep-gpt-pt-de 271.00
ep-gpt-fi-sv 270.00
ep-gpt-de-el 269.00
ep-gpt-sv-el 268.01
ep-gpt-nl-it 268.00
ep-gpt-sv-de 268.00
ep-gpt-it-da 267.00
ep-gpt-it-nl 266.00
flores-gpt-es-el 264.90
ep-gpt-nl-fr 264.00
flores-gpt-fr-el 263.81
ep-gpt-fi-el 263.32
flores-gpt-el-da 263.09
ep-gpt-nl-da 263.00
flores-gpt-sv-el 261.04
flores-gpt-fi-el 259.55
ep-gpt-el-fi 257.00
flores-gpt-pt-de 255.99
ep-gpt-es-el 255.00
ep-gpt-de-en 252.00
flores-gpt-pt-fi 251.67
ep-gpt-da-nl 245.00
ep-g

In [28]:
mean = sum(times)/len(times)
print(f'Mean: {mean:.2f}s')
print(f'Max: {max(times):.2f}s')
print(f'Min: {min(times):.2f}s')

Mean: 192.07s
Max: 301.00s
Min: 63.00s


## Alignment Scheduling
* We can somewhat guess which translations have to be aligned again by checking if the number of outlines != 400

In [30]:
from os.path import join
import json
with open(join('translations', 'info.json'), 'r') as f:
    prefix2file = json.load(f)

num = 0
for prefix, info in prefix2file.items():
    outlines = info['log']['out_lines']
    if outlines != 400:
        print(prefix, outlines)
        num += 1

print(f'{num} pairs need to be re-aligned')

ep-gpt-da-it 399
ep-gpt-de-fi 399
ep-gpt-el-it 399
ep-gpt-el-nl 399
ep-gpt-es-el 399
ep-gpt-es-nl 399
ep-gpt-fi-it 399
ep-gpt-fi-nl 399
ep-gpt-pt-fi 401
ep-gpt-sv-fi 399
flores-gpt-es-el 402
flores-gpt-fi-da 402
flores-gpt-fi-el 402
flores-gpt-fi-es 402
flores-gpt-fi-nl 402
flores-gpt-fi-pt 402
flores-gpt-it-fr 402
ep-gpt-it-fi 399
18 pairs need to be re-aligned
